In [1]:
import pandas as pd
from pathlib import Path
import pymc as pm
import plotly.express as px
import numpy as np
import pandas as pd

from estival.model import BayesianCompartmentalModel
import estival.priors as esp
import estival.targets as est
from estival.wrappers import pymc as epm
from tbdynamics import model
from tbdynamics.utils import round_sigfig
from tbdynamics.inputs import fixed_parameters
import plotly.graph_objects as go

WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


## Define variables

In [2]:
PROJECT_PATH = Path().resolve()
DATA_PATH = PROJECT_PATH / 'data'
pd.options.plotting.backend = "plotly"

time_start = 1800.0
time_end = 2023.0
time_step = 0.1

compartments = [
    "susceptible",
    "early_latent",
    "late_latent",
    "infectious",
    "on_treatment",
    "recovered",
]

latent_compartments = [
    "early_latent",
    "late_latent",
]
infectious_compartments = [
    "infectious",
    "on_treatment",
]


age_strata = [0,5,15,35,50,70] 
matrix = np.ones((6, 6)) # Homo mixing

## Define Model

### Base model

In [3]:
tb_model = model.build_model(
    compartments,
    latent_compartments,
    infectious_compartments,
    age_strata,
    time_start,
    time_end,
    time_step,
    fixed_parameters,
    matrix
)

### Params and calibration targets

In [4]:
params = {
    "contact_rate": 0.009414102898074345,
    "start_population_size": 23000000,
    "progression_multiplier": 1.1,
    "rr_infection_latent": 0.2,
    "rr_infection_recovered": 0.2,
    "infect_death_rate_unstratified": 0.21,
    "smear_positive_death_rate":0.364337776897486,
    "smear_negative_death_rate": 0.027588310343242016, 
    "smear_positive_self_recovery": 0.20344728302826143,
    "smear_negative_self_recovery": 0.22723824998716693,
    "seed_time": 1960.0,
    "seed_duration": 1.0,
}


priors = [
    esp.UniformPrior("start_population_size", (100000, 30000000)),
    esp.UniformPrior("contact_rate", (0, 10)),  # multiplied with eigenvalue
    esp.UniformPrior("rr_infection_latent", (0.2, 0.5)),
    esp.UniformPrior("rr_infection_recovered", (0.1, 0.5)),
    esp.UniformPrior("progression_multiplier", (0.1, 10.0)),
    esp.UniformPrior("seed_time", [1800.0, 1960.0]),
    esp.UniformPrior("seed_duration", [1.0, 5.0]),
]

pop = pd.Series(
    {
        2009: 86025000,
        2019: 96484000,

    }
)

targets = [
    est.NormalTarget("total_population", pop, stdev=100000),
]

calibration_model = BayesianCompartmentalModel(tb_model, params, priors, targets)

### Running Calibration

In [5]:
with pm.Model() as pmc_model:
    start_params = {k: np.clip(v, *calibration_model.priors[k].bounds(0.99)) for k, v in params.items() if k in calibration_model.priors}
    variables = epm.use_model(calibration_model)
    map_params = pm.find_MAP(start=start_params, vars=variables, maxeval= 5000,include_transformed=False)
    map_params = {k: float(v) for k, v in map_params.items()}
    print('Best calibration parameters found:')
for i_param, param in enumerate(map_params):
    print(f'   {param}: {round_sigfig(map_params[param], 4)} (within bound {priors[i_param].bounds()}')


Best calibration parameters found:
   start_population_size: 1069000.0 (within bound (100000.0, 30000000.0)
   contact_rate: 4.724 (within bound (0.0, 10.0)
   rr_infection_latent: 0.3417 (within bound (0.2, 0.5)
   rr_infection_recovered: 0.4934 (within bound (0.1, 0.5)
   progression_multiplier: 9.529 (within bound (0.1, 10.0)
   seed_time: 1960.0 (within bound (1800.0, 1960.0)
   seed_duration: 2.889 (within bound (1.0, 5.0)


In [6]:
params.update(map_params)
tb_model.run(params)
derived_df_0 = tb_model.get_derived_outputs_df()

### Population output

In [7]:
fig2_1 = px.line(
    derived_df_0,
    x=derived_df_0.index,
    y="total_population",
)
fig2_2 = px.scatter(x= pop.index, y = pop.values)
fig2_2.update_traces(marker=dict(color="red"))
fig2_3 = go.Figure(
    data=fig2_1.data + fig2_2.data,
)
fig2_3.update_layout(
    title="Modelled vs Data", title_x=0.5, xaxis_title="Year", yaxis_title="Population"
)
fig2_3.show()

In [8]:
derived_df_0[[f'total_populationXage_{i}' for i in age_strata]].plot.area(title='Modelled populatation by age group')

In [9]:
derived_df_0

,infectious_population_size,latent_population_size,percentage_latent,prevalence_infectious,total_population,total_populationXage_0,total_populationXage_15,total_populationXage_35,total_populationXage_5,total_populationXage_50,total_populationXage_70
1800.0,0.0,0.0,0.0,0.0,1.069497e+06,1.782495e+05,1.782495e+05,1.782495e+05,1.782495e+05,1.782495e+05,1.782495e+05
1800.1,0.0,0.0,0.0,0.0,1.070663e+06,1.782853e+05,1.790039e+05,1.777601e+05,1.799447e+05,1.781355e+05,1.775340e+05
1800.2,0.0,0.0,0.0,0.0,1.071840e+06,1.783249e+05,1.797708e+05,1.772782e+05,1.816231e+05,1.780190e+05,1.768244e+05
1800.3,0.0,0.0,0.0,0.0,1.073028e+06,1.783682e+05,1.805498e+05,1.768040e+05,1.832849e+05,1.779003e+05,1.761206e+05
1800.4,0.0,0.0,0.0,0.0,1.074225e+06,1.784152e+05,1.813409e+05,1.763373e+05,1.849303e+05,1.777793e+05,1.754225e+05
...,...,...,...,...,...,...,...,...,...,...,...
2022.6,0.0,0.0,0.0,0.0,9.727834e+07,7.826448e+06,2.843140e+07,1.883839e+07,1.515419e+07,1.721557e+07,9.812351e+06
2022.7,0.0,0.0,0.0,0.0,9.734364e+07,7.831352e+06,2.843605e+07,1.884957e+07,1.515802e+07,1.723799e+07,9.830656e+06
2022.8,0.0,0.0,0.0,0.0,9.740889e+07,7.836266e+06,2.844073e+07,1.886070e+07,1.516191e+07,1.726035e+07,9.848947e+06
2022.9,0.0,0.0,0.0,0.0,9.747409e+07,7.841190e+06,2.844541e+07,1.887177e+07,1.516586e+07,1.728265e+07,9.867223e+06
